<a href="https://colab.research.google.com/github/ddv246/DSBA-Project/blob/master/Yellow_Taxi_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ddv246/DSBA-Project/blob/master/Yellow_Taxi_Project.ipynb)

In [106]:
#If opening in colab run this cell
!git clone https://github.com/ddv246/DSBA-Project
%cd DSBA-Project/YellowTaxi

Cloning into 'DSBA-Project'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 34 (delta 12), reused 24 (delta 5), pack-reused 0
Unpacking objects: 100% (34/34), done.
/content/DSBA-Project/YellowTaxi/DSBA-Project/YellowTaxi/DSBA-Project/YellowTaxi/DSBA-Project/YellowTaxi


In [0]:
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium
from datetime import datetime
import time
import seaborn as sns
import os
import math
import xgboost as xgb
import matplotlib


In [0]:
#import glob


#all_files1 = []
#for name in glob.glob('YellowTaxi/*'):
 #   all_files1.append(name)

#li = []


#for filename in all_files1:
 #   df = pd.read_csv(filename, index_col=None, header=0)
  #  li.append(df)

#frame = pd.concat(li, axis=0, ignore_index=True)



In [108]:
frame=pd.read_csv("2015_sample.csv")
frame

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-10-01 00:00:00,2015-10-01 00:05:48,1,1.10,-73.935516,40.761238,1,N,-73.944351,40.754578,2,6.00,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2015-10-01 00:00:00,2015-10-01 00:00:00,1,7.68,-73.989937,40.743439,1,N,-73.986687,40.689129,2,27.50,0.0,0.5,0.00,0.0,0.3,28.30
2,2,2015-10-01 00:00:00,2015-10-01 00:00:00,2,2.53,-73.987328,40.720020,1,N,-73.999084,40.744381,1,12.50,0.0,0.5,2.00,0.0,0.3,15.30
3,2,2015-10-01 00:00:00,2015-10-01 00:00:00,0,1.20,-73.953758,40.743385,5,N,-73.930008,40.736622,2,25.26,0.0,0.5,0.00,0.0,0.3,26.06
4,1,2015-10-01 00:00:01,2015-10-01 00:16:19,1,3.80,-73.984016,40.755222,1,N,-73.959869,40.801323,1,15.50,0.5,0.5,3.00,0.0,0.3,19.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2,2015-10-01 09:33:05,2015-10-01 09:55:19,1,5.00,-73.960625,40.761410,1,N,-74.002640,40.728294,1,20.50,0.0,0.5,4.26,0.0,0.3,25.56
99995,1,2015-10-01 09:33:06,2015-10-01 09:46:03,1,1.00,-74.008865,40.715946,1,N,-73.997917,40.720467,1,9.00,0.0,0.5,2.90,0.0,0.3,12.70
99996,1,2015-10-01 09:33:06,2015-10-01 09:57:39,1,2.60,-73.981049,40.729874,1,N,-73.991631,40.754829,1,16.00,0.0,0.5,3.36,0.0,0.3,20.16
99997,2,2015-10-01 09:33:06,2015-10-01 09:52:57,5,0.99,-73.975304,40.759300,1,N,-73.964859,40.769749,1,13.00,0.0,0.5,2.76,0.0,0.3,16.56


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1177 entries, 15 to 99875
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               1177 non-null   int64         
 1   tpep_pickup_datetime   1177 non-null   datetime64[ns]
 2   tpep_dropoff_datetime  1177 non-null   datetime64[ns]
 3   passenger_count        1177 non-null   int64         
 4   trip_distance          1177 non-null   float64       
 5   pickup_longitude       1177 non-null   float64       
 6   pickup_latitude        1177 non-null   float64       
 7   dropoff_longitude      1177 non-null   float64       
 8   dropoff_latitude       1177 non-null   float64       
 9   payment_type           1177 non-null   int64         
 10  fare_amount            1177 non-null   float64       
 11  extra                  1177 non-null   float64       
 12  mta_tax                1177 non-null   float64       
 13  t

In [110]:
df.isnull().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
pickup_longitude         0
pickup_latitude          0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
trip_duration            0
Trip_duration            0
dtype: int64

In [0]:
df=frame.dropna()


In [0]:
del df['RatecodeID']

In [0]:
del df ['store_and_fwd_flag']

In [0]:
df.drop(df[df.trip_distance ==0.00].index, inplace=True)

In [115]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-10-01 00:00:00,2015-10-01 00:05:48,1,1.10,-73.935516,40.761238,-73.944351,40.754578,2,6.00,0.5,0.5,0.00,0.0,0.3,7.30
1,2,2015-10-01 00:00:00,2015-10-01 00:00:00,1,7.68,-73.989937,40.743439,-73.986687,40.689129,2,27.50,0.0,0.5,0.00,0.0,0.3,28.30
2,2,2015-10-01 00:00:00,2015-10-01 00:00:00,2,2.53,-73.987328,40.720020,-73.999084,40.744381,1,12.50,0.0,0.5,2.00,0.0,0.3,15.30
3,2,2015-10-01 00:00:00,2015-10-01 00:00:00,0,1.20,-73.953758,40.743385,-73.930008,40.736622,2,25.26,0.0,0.5,0.00,0.0,0.3,26.06
4,1,2015-10-01 00:00:01,2015-10-01 00:16:19,1,3.80,-73.984016,40.755222,-73.959869,40.801323,1,15.50,0.5,0.5,3.00,0.0,0.3,19.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2,2015-10-01 09:33:05,2015-10-01 09:55:19,1,5.00,-73.960625,40.761410,-74.002640,40.728294,1,20.50,0.0,0.5,4.26,0.0,0.3,25.56
99995,1,2015-10-01 09:33:06,2015-10-01 09:46:03,1,1.00,-74.008865,40.715946,-73.997917,40.720467,1,9.00,0.0,0.5,2.90,0.0,0.3,12.70
99996,1,2015-10-01 09:33:06,2015-10-01 09:57:39,1,2.60,-73.981049,40.729874,-73.991631,40.754829,1,16.00,0.0,0.5,3.36,0.0,0.3,20.16
99997,2,2015-10-01 09:33:06,2015-10-01 09:52:57,5,0.99,-73.975304,40.759300,-73.964859,40.769749,1,13.00,0.0,0.5,2.76,0.0,0.3,16.56


### **1. Checking uniqueness of ID**

In [116]:
print("Number of ids in the train dataset: ", len(df["VendorID"]))
print("Number of unique ids in the train dataset: ", len(pd.unique(df["VendorID"])), "\n")

Number of ids in the train dataset:  99174
Number of unique ids in the train dataset:  2 



**Description**

Vendor_ID takes on only two values , 1 or 2. If the hypothesis is right and the values in the Vendor_ID column represent the data from two different taxi companies. 

### **2.Checking if drop off time is earlier than pick up time**

In [117]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
(df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).describe()

count                     99174
mean     0 days 00:15:18.138715
std      0 days 00:47:51.121904
min           -1 days +23:58:44
25%             0 days 00:06:21
50%             0 days 00:10:43
75%             0 days 00:17:34
max             0 days 23:59:17
dtype: object

### **Removing rows where dropoff time is earlier than pickup time**

In [118]:
import numpy as np

df=df[(df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]) /np.timedelta64(1, 'D') > 0.0]
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-10-01 00:00:00,2015-10-01 00:05:48,1,1.10,-73.935516,40.761238,-73.944351,40.754578,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30
4,1,2015-10-01 00:00:01,2015-10-01 00:16:19,1,3.80,-73.984016,40.755222,-73.959869,40.801323,1,15.5,0.5,0.5,3.00,0.0,0.3,19.80
5,1,2015-10-01 00:00:01,2015-10-01 00:13:41,1,3.10,-73.975296,40.751396,-73.970924,40.785984,1,12.5,0.5,0.5,1.00,0.0,0.3,14.80
6,1,2015-10-01 00:00:01,2015-10-01 00:21:23,1,4.50,-73.997078,40.722290,-73.960472,40.761517,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80
7,2,2015-10-01 00:00:01,2015-10-01 00:05:50,1,0.84,-73.998711,40.734756,-74.003082,40.742546,1,6.0,0.5,0.5,0.70,0.0,0.3,8.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2,2015-10-01 09:33:05,2015-10-01 09:55:19,1,5.00,-73.960625,40.761410,-74.002640,40.728294,1,20.5,0.0,0.5,4.26,0.0,0.3,25.56
99995,1,2015-10-01 09:33:06,2015-10-01 09:46:03,1,1.00,-74.008865,40.715946,-73.997917,40.720467,1,9.0,0.0,0.5,2.90,0.0,0.3,12.70
99996,1,2015-10-01 09:33:06,2015-10-01 09:57:39,1,2.60,-73.981049,40.729874,-73.991631,40.754829,1,16.0,0.0,0.5,3.36,0.0,0.3,20.16
99997,2,2015-10-01 09:33:06,2015-10-01 09:52:57,5,0.99,-73.975304,40.759300,-73.964859,40.769749,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56


### **Creating a new column called Trip_duration.**

In [119]:
df['Trip_duration']=(df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]) /np.timedelta64(1, 'm')
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Trip_duration
0,1,2015-10-01 00:00:00,2015-10-01 00:05:48,1,1.10,-73.935516,40.761238,-73.944351,40.754578,2,6.0,0.5,0.5,0.00,0.0,0.3,7.30,5.800000
4,1,2015-10-01 00:00:01,2015-10-01 00:16:19,1,3.80,-73.984016,40.755222,-73.959869,40.801323,1,15.5,0.5,0.5,3.00,0.0,0.3,19.80,16.300000
5,1,2015-10-01 00:00:01,2015-10-01 00:13:41,1,3.10,-73.975296,40.751396,-73.970924,40.785984,1,12.5,0.5,0.5,1.00,0.0,0.3,14.80,13.666667
6,1,2015-10-01 00:00:01,2015-10-01 00:21:23,1,4.50,-73.997078,40.722290,-73.960472,40.761517,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80,21.366667
7,2,2015-10-01 00:00:01,2015-10-01 00:05:50,1,0.84,-73.998711,40.734756,-74.003082,40.742546,1,6.0,0.5,0.5,0.70,0.0,0.3,8.00,5.816667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2,2015-10-01 09:33:05,2015-10-01 09:55:19,1,5.00,-73.960625,40.761410,-74.002640,40.728294,1,20.5,0.0,0.5,4.26,0.0,0.3,25.56,22.233333
99995,1,2015-10-01 09:33:06,2015-10-01 09:46:03,1,1.00,-74.008865,40.715946,-73.997917,40.720467,1,9.0,0.0,0.5,2.90,0.0,0.3,12.70,12.950000
99996,1,2015-10-01 09:33:06,2015-10-01 09:57:39,1,2.60,-73.981049,40.729874,-73.991631,40.754829,1,16.0,0.0,0.5,3.36,0.0,0.3,20.16,24.550000
99997,2,2015-10-01 09:33:06,2015-10-01 09:52:57,5,0.99,-73.975304,40.759300,-73.964859,40.769749,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56,19.850000


In [120]:
df.Trip_duration.describe()

count    99161.000000
mean        15.304331
std         47.854843
min          0.016667
25%          6.366667
50%         10.716667
75%         17.566667
max       1439.283333
Name: Trip_duration, dtype: float64

### **3.Checking if passenger_count is less than 0**

In [121]:
df.passenger_count.describe()

count    99161.000000
mean         1.613195
std          1.321662
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          6.000000
Name: passenger_count, dtype: float64

In [122]:
df=df[df.passenger_count >= 1]
df.passenger_count.describe()

count    99160.000000
mean         1.613211
std          1.321659
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          6.000000
Name: passenger_count, dtype: float64

#### **Description:** 
A trip cannot happen without passenger, therefore, removing rows with passenger_count < 1

### **4. Checking if Trip distance is less than 0.1 miles.**

In [123]:
df.trip_distance.describe()

count    99160.000000
mean         3.098804
std          3.825100
min          0.010000
25%          1.020000
50%          1.800000
75%          3.410000
max        225.800000
Name: trip_distance, dtype: float64

In [124]:
df[df.trip_distance==0.1]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Trip_duration
912,1,2015-10-14 10:25:52,2015-10-14 10:33:07,1,0.1,-73.979515,40.743816,-73.977898,40.746399,1,6.0,0.0,0.5,0.00,0.00,0.3,6.80,7.250000
1358,1,2015-10-14 10:27:19,2015-10-14 10:31:45,1,0.1,-73.976395,40.752182,-73.976395,40.752182,1,4.5,0.0,0.5,1.32,0.00,0.3,6.62,4.433333
1684,1,2015-10-14 10:28:24,2015-10-14 10:29:10,1,0.1,-73.976013,40.791401,-73.975189,40.790958,2,2.5,0.0,0.5,0.00,0.00,0.3,3.30,0.766667
1985,1,2015-10-14 10:29:27,2015-10-14 11:06:22,1,0.1,-73.888039,40.767563,-73.973091,40.757507,2,2.5,0.0,0.5,0.00,5.54,0.3,8.84,36.916667
2434,1,2015-10-14 10:30:54,2015-10-14 10:32:25,4,0.1,-73.978043,40.752914,-73.977478,40.751820,1,3.0,0.0,0.5,0.75,0.00,0.3,4.55,1.516667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95590,1,2015-10-01 09:19:49,2015-10-01 09:25:31,1,0.1,-73.951530,40.791168,-73.953491,40.791164,2,5.0,0.0,0.5,0.00,0.00,0.3,5.80,5.700000
96738,2,2015-10-01 09:23:11,2015-10-01 09:23:58,1,0.1,-74.004303,40.707561,-74.002876,40.708431,2,2.5,0.0,0.5,0.00,0.00,0.3,3.30,0.783333
97043,1,2015-10-01 09:24:01,2015-10-01 09:25:13,1,0.1,-73.976715,40.759525,-73.977608,40.758144,2,3.0,0.0,0.5,0.00,0.00,0.3,3.80,1.200000
97280,1,2015-10-01 09:24:44,2015-10-01 09:28:56,1,0.1,-73.977715,40.764503,-73.974709,40.763504,1,4.5,0.0,0.5,1.05,0.00,0.3,6.35,4.200000


In [125]:
df=df[df.trip_distance >= 0.1]
df.trip_distance.describe()

count    99021.000000
mean         3.103100
std          3.826064
min          0.100000
25%          1.030000
50%          1.800000
75%          3.420000
max        225.800000
Name: trip_distance, dtype: float64

**Description:**
Trip distance cannot be less than 0.1 miles ( 0.1 miles as considering a passenger might hops in to a taxi but hops off due to traffic). Thus removing all such rows 

### **5.Checking if Payment type is invalid.**

In [126]:
df.payment_type.describe()

count    99021.000000
mean         1.328163
std          0.481902
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          4.000000
Name: payment_type, dtype: float64

**Description:**

Payment types: 

A numeric code signifying how the passenger paid for the trip.

1= Credit card

2= Cash

3= No charge

4= Dispute

### **6. Checking if Latitute and Longtitude fall within NYC** 

In [127]:
#detecting the pickups latitude and longitudes which are outside NYC.
out_nyc=df[((df.pickup_latitude <= 40.5774) | (df.pickup_longitude <= -74.15) | (df.pickup_latitude >= 40.9176) | (df.pickup_longitude >= -73.7004))]
out_nyc

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Trip_duration
15,1,2015-10-01 00:00:03,2015-10-01 00:03:57,1,0.60,0.0,0.0,0.0,0.0,1,4.5,0.5,0.5,1.16,0.0,0.3,6.96,3.900000
185,2,2015-10-14 10:23:18,2015-10-14 10:32:23,1,1.33,0.0,0.0,0.0,0.0,1,8.0,0.0,0.5,1.76,0.0,0.3,10.56,9.083333
265,1,2015-10-14 10:23:36,2015-10-14 10:32:39,1,0.90,0.0,0.0,0.0,0.0,1,7.0,0.0,0.5,1.55,0.0,0.3,9.35,9.050000
358,2,2015-10-14 10:23:56,2015-10-14 10:39:58,1,2.66,0.0,0.0,0.0,0.0,2,12.5,0.0,0.5,0.00,0.0,0.3,13.30,16.033333
456,2,2015-10-14 10:24:18,2015-10-14 10:26:49,1,0.18,0.0,0.0,0.0,0.0,2,3.5,0.0,0.5,0.00,0.0,0.3,4.30,2.516667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99707,1,2015-10-01 09:32:16,2015-10-01 09:43:02,1,2.00,0.0,0.0,0.0,0.0,1,9.5,0.0,0.5,2.05,0.0,0.3,12.35,10.766667
99713,2,2015-10-01 09:32:16,2015-10-01 09:41:48,1,0.88,0.0,0.0,0.0,0.0,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80,9.533333
99715,2,2015-10-01 09:32:16,2015-10-01 09:47:31,1,1.60,0.0,0.0,0.0,0.0,1,11.0,0.0,0.5,2.95,0.0,0.3,14.75,15.250000
99799,2,2015-10-01 09:32:29,2015-10-01 09:46:00,1,0.91,0.0,0.0,0.0,0.0,2,9.0,0.0,0.5,0.00,0.0,0.3,9.80,13.516667


In [128]:
m = folium.Map(location = [40.5774, -73.7004], tiles = "Stamen Toner")

outside_pickups = out_nyc.head(25000)

for i,j in outside_pickups.iterrows():
    if j["pickup_latitude"] != 0:
        folium.Marker([j["pickup_latitude"], j["pickup_longitude"]]).add_to(m)
m

**Description:** 

As you can see in the above map, there are many erroneous data points. Some of which are in other state or outside of NYC.  All these erroneous data points are removed. 